<a href="https://colab.research.google.com/github/Wuwei0-1/Dodeca/blob/main/Websitev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bcrypt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 10.3 MB/s eta 0:00:00


In [ ]:
pip install pyngrok

In [ ]:
!pip install nest_asyncio

In [ ]:
# Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual token from https://dashboard.ngrok.com/get-started/your-authtoken
from pyngrok import ngrok
ngrok.set_auth_token("36pRyZqLQvS6cyI6eFvJynKtLov_7ninaQrW6gCTcb1VGNpf4")

In [ ]:
import os

if os.path.exists('secure_vault.db'):
    os.remove('secure_vault.db')
    print('secure_vault.db deleted successfully.')
else:
    print('secure_vault.db does not exist.')


secure_vault.db does not exist.


In [ ]:
from pyngrok import ngrok

# Terminate any previous ngrok tunnels
ngrok.kill()

# Open a ngrok tunnel to the FastAPI app, which is running on port 8000
public_url = ngrok.connect(8000)

# Update the global variable in the main app cell with the current ngrok URL
global NGROK_PUBLIC_URL
NGROK_PUBLIC_URL = public_url.public_url # Correctly access the public_url attribute

print(f"Public URL: {NGROK_PUBLIC_URL}")

Public URL: https://meditatingly-unrefunding-keyla.ngrok-free.dev


After setting your authtoken in the cell above, you will need to re-run the `ngrok.connect()` cell (`b2cf8bdc`) to establish the tunnel.

After running the above cell, you should see a clear `Public URL:` printed in the output. This is the `https://` link you need to use to access your application from your iPad. Make sure to keep the Colab tab open, as closing it will terminate the `ngrok` tunnel.

In [ ]:
import gradio as gr
import sqlite3
import datetime
import os
import secrets
from cryptography.fernet import Fernet
import bcrypt
import nest_asyncio

# --- WEB SERVER LIBRARIES ---
from fastapi import FastAPI, Request
from fastapi.responses import RedirectResponse, HTMLResponse
from starlette.middleware.sessions import SessionMiddleware
from authlib.integrations.starlette_client import OAuth
import uvicorn

# --- CONFIGURATION ---
DB_NAME = "secure_vault.db"
KEY_FILE = "secret.key"
SECRET_KEY = secrets.token_hex(32) # For session cookies

# !!! PASTE YOUR GOOGLE KEYS HERE !!!
GOOGLE_CLIENT_ID = "969341989582-lfutryj821t58p07997hgc2kv.apps.googleusercontent.com"
GOOGLE_CLIENT_SECRET = "GOCSPX-HHmCoY75wGEBWMW_yrIxRSJComf"

# --- ENCRYPTION SETUP ---
if not os.path.exists(KEY_FILE):
    with open(KEY_FILE, "wb") as kf:
        kf.write(Fernet.generate_key())
with open(KEY_FILE, "rb") as kf:
    cipher = Fernet(kf.read())

# --- DATABASE ENGINE ---
def init_db():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (username TEXT PRIMARY KEY, password_hash BLOB, role TEXT, auth_type TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS inventions
                 (id TEXT PRIMARY KEY, owner TEXT, title TEXT, filename TEXT, data BLOB, timestamp TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS access
                 (invention_id TEXT, username TEXT)''')
    conn.commit()
    conn.close()

init_db()

# --- FASTAPI & OAUTH SETUP ---
app = FastAPI()
app.add_middleware(SessionMiddleware, secret_key=SECRET_KEY)

oauth = OAuth()
oauth.register(
    name='google',
    client_id=GOOGLE_CLIENT_ID,
    client_secret=GOOGLE_CLIENT_SECRET,
    server_metadata_url='https://accounts.google.com/.well-known/openid-configuration',
    client_kwargs={'scope': 'openid email profile'}
)

# --- LOGIN PAGE HTML ---
# This custom page replaces the standard Gradio login
LOGIN_HTML = """
<!DOCTYPE html>
<html>
<head>
    <title>SecureVault Login/Register</title>
    <style>
        body { font-family: sans-serif; background: #f0f2f6; display: flex; justify-content: center; align-items: flex-start; height: 100vh; padding-top: 50px; }
        .container { display: flex; gap: 30px; }
        .card { background: white; padding: 2rem; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); width: 350px; text-align: center; }
        input, select { width: 90%; padding: 10px; margin: 10px 0; border: 1px solid #ddd; border-radius: 5px; }
        button { width: 100%; padding: 10px; background: #10b981; color: white; border: none; border-radius: 5px; cursor: pointer; font-weight: bold; }
        button:hover { background: #059669; }
        .google-btn { background: #4285F4; margin-top: 10px; }
        .google-btn:hover { background: #357ae8; }
        .divider { margin: 20px 0; color: #888; font-size: 0.8rem; }
        h2 { margin-bottom: 25px; }
        h3 { margin-top: 0; margin-bottom: 15px; color: #333; }
    </style>
</head>
<body>
    <div class=\"container\">
        <div class=\"card\">
            <h2>🛡️ SecureVault</h2>
            <h3>Sign In</h3>

            <!-- STANDARD LOGIN -->
            <form action=\"/login/password\" method=\"post\">
                <input type=\"text\" name=\"username\" placeholder=\"Username\" required>
                <input type=\"password\" name=\"password\" placeholder=\"Password\" required>
                <button type=\"submit\">Sign In</button>
            </form>

            <div class=\"divider\">OR</div>

            <!-- GOOGLE LOGIN -->
            <a href=\"/login/google\">
                <button class=\"google-btn\">Sign in with Gmail</button>
            </a>
        </div>

        <div class=\"card\">
            <h2>🛡️ SecureVault</h2>
            <h3>Register New Account</h3>

            <!-- REGISTRATION FORM -->
            <form action=\"/register\" method=\"post\">
                <input type=\"text\" name=\"username\" placeholder=\"Choose a Username\" required>
                <input type=\"password\" name=\"password\" placeholder=\"Create a Password\" required>
                <select name=\"role\" required>
                    <option value=\"\">Select Account Type</option>
                    <option value=\"Inventor\">Inventor</option>
                    <option value=\"Reviewer\">Reviewer</option>
                    <option value=\"Collaborator\">Collaborator</option>
                    <option value=\"Admin\">Admin</option>
                </select>
                <button type=\"submit\">Register</button>
            </form>
        </div>
    </div>
</body>
</html>
"""

# --- ROUTES ---

@app.get("/", response_class=HTMLResponse)
async def root(request: Request):
    # If logged in, go to App. If not, go to Login.
    user = request.session.get('user')
    if user:
        return RedirectResponse(url="/vault")
    return HTMLResponse(content=LOGIN_HTML)

# Global variable to store the ngrok public URL
global NGROK_PUBLIC_URL

@app.get("/login/google")
async def login_google(request: Request):
    global NGROK_PUBLIC_URL
    if not NGROK_PUBLIC_URL:
        return HTMLResponse("❌ ngrok public URL not set. Please ensure the ngrok tunnel cell (`2bb5f35b`) has been run and provided a URL. <a href='/'>Try Again</a>")

    redirect_uri = f"{NGROK_PUBLIC_URL}/auth/google"
    return await oauth.google.authorize_redirect(request, redirect_uri)

@app.get("/auth/google")
async def auth_google(request: Request):
    token = await oauth.google.authorize_access_token(request)
    user_info = token.get('userinfo')
    email = user_info['email']

    # Auto-register Google User if not exists
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT * FROM users WHERE username=?", (email,))
    if not c.fetchone():
        # Default role for Google users is 'Inventor' (Viewer), they can upgrade later
        c.execute("INSERT INTO users VALUES (?, ?, ?, ?)", (email, b"GOOGLE_AUTH", "Inventor", "Google"))
        conn.commit()
    conn.close()

    request.session['user'] = email
    return RedirectResponse(url="/vault")

# Simple password login handler (needs form parsing in real prod, simplified here)
# For this demo, we assume the user logs in via the Gradio UI or Google.
# But to make the HTML form work, we need a small endpoint:
from starlette.datastructures import FormData
@app.post("/login/password")
async def login_password(request: Request):
    form = await request.form()
    username = form.get("username")
    password = form.get("password")

    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password_hash FROM users WHERE username=?", (username,))
    record = c.fetchone()
    conn.close()

    if record and bcrypt.checkpw(password.encode(), record[0]):
        request.session['user'] = username
        return RedirectResponse(url="/vault")
    else:
        return HTMLResponse("❌ Invalid Credentials. <a href='/'>Try Again</a>")

@app.post("/register")
async def register_user(request: Request):
    form = await request.form()
    print(f"DEBUG: Register function called for username: {form.get('username')}")
    username = form.get("username")
    password = form.get("password")
    role = form.get("role")

    if not username or not password or not role:
        return HTMLResponse("❌ Please fill in all fields for registration. <a href='/'>Try Again</a>")

    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    try:
        # Check if username already exists
        c.execute("SELECT * FROM users WHERE username=?", (username,))
        if c.fetchone():
            conn.close()
            return HTMLResponse("❌ Username already exists. Please choose a different one. <a href='/'>Try Again</a>")

        # --- New logic for single Admin account ---
        if role == "Admin":
            c.execute("SELECT * FROM users WHERE role=?", ("Admin",))
            if c.fetchone():
                conn.close()
                return HTMLResponse("❌ An Admin account already exists. Only one Admin account is allowed. <a href='/'>Try Again</a>")
        # --- End new logic ---

        # Hash the password
        hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt())

        c.execute("INSERT INTO users VALUES (?, ?, ?, ?)", (username, hashed_password, role, "Local"))
        conn.commit()
        conn.close()
        # After successful registration, log them in or redirect to login page
        request.session['user'] = username # Automatically log in
        print(f"DEBUG: Redirecting registered user '{username}' to /vault")
        return RedirectResponse(url="/vault")
    except sqlite3.IntegrityError:
        conn.close()
        return HTMLResponse("❌ Error registering user. Username might already exist. <a href='/'>Try Again</a>")
    except Exception as e:
        conn.close()
        return HTMLResponse(f"❌ An unexpected error occurred: {e} <a href='/'>Try Again</a>")

@app.get("/logout")
async def logout(request: Request):
    request.session.pop('user', None)
    return RedirectResponse(url="/")

# --- GRADIO APP LOGIC (Protected) ---

def get_current_user(request: gr.Request):
    # This magic function grabs the user from the FastAPI session
    user = request.session.get('user')
    if not user:
        raise gr.Error("⛔ Session Expired. Please refresh and log in.")
    return user

def upload_file(req: gr.Request, title, file_obj):
    user = get_current_user(req)
    if not file_obj: return "❌ No file."

    with open(file_obj, "rb") as f:
        encrypted = cipher.encrypt(f.read())

    unique_id = secrets.token_hex(4)
    fname = os.path.basename(file_obj)

    conn = sqlite3.connect(DB_NAME)
    conn.execute("INSERT INTO inventions VALUES (?, ?, ?, ?, ?, ?)",
                 (unique_id, user, title, fname, encrypted, str(datetime.datetime.now())))
    conn.execute("INSERT INTO access VALUES (?, ?)", (unique_id, user))
    conn.commit()
    conn.close()
    return f"🔒 Encrypted! ID: {unique_id}"

def my_files(req: gr.Request):
    user = get_current_user(req)
    conn = sqlite3.connect(DB_NAME)
    data = conn.execute("SELECT id, title, timestamp FROM inventions WHERE owner=?", (user,)).fetchall()
    conn.close()
    return "\n".join([f"📄 {row[1]} (ID: {row[0]})" for row in data])

# --- GRADIO UI ---
with gr.Blocks(theme=gr.themes.Soft(), title="Vault") as vault_ui:
    gr.Markdown(f"# 🏦 The Vault")

    with gr.Tab("My Dashboard"):
        gr.Markdown("### Welcome")
        who_btn = gr.Button("Who am I?")
        who_out = gr.Textbox()
        who_btn.click(fn=lambda r: f"Logged in as: {get_current_user(r)}", inputs=None, outputs=who_out)

        gr.Markdown("### Upload")
        u_title = gr.Textbox(label="Title")
        u_file = gr.File()
        u_btn = gr.Button("Encrypt & Store")
        u_out = gr.Textbox()
        u_btn.click(upload_file, [u_title, u_file], u_out)

    with gr.Tab("My Inventions"):
        ref_btn = gr.Button("Refresh List")
        ref_out = gr.Textbox(lines=10)
        ref_btn.click(my_files, None, ref_out)

    logout_link = gr.Markdown("[🚪 Sign Out](/logout)")

# --- MOUNTING ---
# This puts Gradio INSIDE FastAPI
app = gr.mount_gradio_app(app, vault_ui, path="/vault")

# --- EXECUTION ---
if __name__ == "__main__":
    print("🚀 Server starting at http://0.0.0.0:8000")
    nest_asyncio.apply() # Apply nest_asyncio to allow running in an existing event loop

    # Configure Uvicorn server explicitly to run in the existing event loop
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)

    import asyncio
    asyncio.create_task(server.serve())
    # Add a small delay to ensure the server starts listening before ngrok connects
    asyncio.create_task(asyncio.sleep(3)) # Sleep for 3 seconds
    print("FastAPI app is now running in the background.")
    print("Access it at: http://localhost:8000 (if running locally)")
    print("Or use the public URL provided by Colab if available.")

/tmp/ipython-input-3892423328.py:274: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="Vault") as vault_ui:
/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1052: UserWarning: Expected 1 arguments for function <function <lambda> at 0x7bfa50762ca0>, received 0.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1056: UserWarning: Expected at least 1 arguments for function <function <lambda> at 0x7bfa50762ca0>, received 0.
  warnings.warn(


new /vault
🚀 Server starting at http://0.0.0.0:8000
FastAPI app is now running in the background.
Access it at: http://localhost:8000 (if running locally)
Or use the public URL provided by Colab if available.


# Task
Test the user registration and sign-in functionalities of the SecureVault application by:
1. Registering a new local user with a unique username, strong password, and 'Inventor' role via the `/register` endpoint and reporting redirection and any errors.
2. Attempting to register users with an existing username and an 'Admin' role (if one exists) to verify error handling.
3. Verifying the newly registered local user's details in the `secure_vault.db` database's `users` table.
4. Signing in with the registered local user's credentials via the `/login/password` endpoint, verifying successful redirection to `/vault`.
5. Attempting to sign in with incorrect local credentials and reporting error messages.
6. Initiating and completing a Google sign-in via `/login/google`, verifying successful redirection to `/vault`.
7. Summarizing the success/failure of each test and the overall accessibility of the application.

## Test Local User Registration

### Subtask:
Attempt to register a new user through the `/register` endpoint. Provide a unique username, a strong password, and select a role (e.g., 'Inventor'). After successful registration, you should be automatically redirected to '/vault'. Report if this redirection occurs or if any error messages are displayed. Then, attempt to register a user with an existing username and an 'Admin' user (if one already exists) to verify error handling for these scenarios.


**Reasoning**:
The subtask requires testing the /register endpoint. First, I need to import the `requests` library to make HTTP requests and then define the base URL using the existing `NGROK_PUBLIC_URL` variable.



In [ ]:
import requests

# Ensure NGROK_PUBLIC_URL is available from the kernel state
print(f"NGROK Public URL: {NGROK_PUBLIC_URL}")

NGROK Public URL: https://meditatingly-unrefunding-keyla.ngrok-free.dev


**Reasoning**:
The next step is to perform the registration tests as specified: register a new user, attempt to register an existing user, and attempt to register a second admin user. This involves constructing payloads, sending POST requests to the `/register` endpoint, and printing the responses to verify the application's behavior.



In [ ]:
base_url = NGROK_PUBLIC_URL

# 3. Register a new 'Inventor' user
print("\n--- Test Case 1: Register new 'Inventor' user ---")
new_username = "test_user_inventor@example.com"
new_password = "StrongPassword123!"
payload_new_user = {
    "username": new_username,
    "password": new_password,
    "role": "Inventor"
}

response_new_user = requests.post(f"{base_url}/register", data=payload_new_user)
print(f"Status Code: {response_new_user.status_code}")
print(f"Response Text: {response_new_user.text}")
if response_new_user.status_code == 302 and "/vault" in response_new_user.headers.get('Location', ''):
    print("\u2705 Success: New 'Inventor' user registered and redirected to /vault.")
else:
    print("\u274C Failure: New 'Inventor' user registration or redirection failed.")

# 4. Attempt to register an existing username
print("\n--- Test Case 2: Attempt to register existing username ---")
payload_existing_user = {
    "username": new_username, # Use the same username
    "password": "AnotherStrongPass456!",
    "role": "Reviewer"
}

response_existing_user = requests.post(f"{base_url}/register", data=payload_existing_user)
print(f"Status Code: {response_existing_user.status_code}")
print(f"Response Text: {response_existing_user.text}")
if "Username already exists" in response_existing_user.text:
    print("\u2705 Success: Correctly handled registration of existing username.")
else:
    print("\u274C Failure: Did not prevent registration of existing username or showed wrong error.")

# 5. Attempt to register a second 'Admin' user
print("\n--- Test Case 3: Attempt to register a second 'Admin' user ---")
second_admin_username = "second_admin@example.com"
payload_second_admin = {
    "username": second_admin_username,
    "password": "AdminPass789!",
    "role": "Admin"
}

response_second_admin = requests.post(f"{base_url}/register", data=payload_second_admin)
print(f"Status Code: {response_second_admin.status_code}")
print(f"Response Text: {response_second_admin.text}")
if "An Admin account already exists" in response_second_admin.text:
    print("\u2705 Success: Correctly prevented registration of a second Admin user.")
else:
    print("\u274C Failure: Allowed registration of a second Admin user or showed wrong error.")


--- Test Case 1: Register new 'Inventor' user ---


KeyboardInterrupt: 